In [1]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import time
import logging
import cssutils as css
css.log.setLevel(logging.CRITICAL)

In [2]:
URL = "https://en.tutiempo.net/climate/{}-{}/ws-{}.html"

In [3]:
cities = {
    'Toronto': '716240',
    'Vancouver': '718920',
    'Montreal':'716270',
    'Ottawa':'716280',
    'St John':'718010',
    'Charlottetown':'717060',
    'Halifax':'713950'
}

In [4]:
def get_style_str(soup):
    for style in soup.find_all('style'):
        decoded_str = style.decode_contents()
        if 'numspan' in decoded_str and 'span.nt' in decoded_str:
            return decoded_str
    return ''

In [5]:
def get_span_dict(soup):
    span_dict = {}
    style_str = get_style_str(soup)
    sheet = css.parseString(style_str)
    for rule in sheet:
        txt = rule.selectorText
        if 'numspan' in txt and 'span.nt' in txt:
            key = txt.split('.')[2].split('::')[0]
            for prop in rule.style:
                if prop.name == 'content':
                    span_dict[key] = prop.value.strip('"')
    return span_dict

In [8]:
css.util

<module 'cssutils.util' from 'C:\\Users\\navee\\miniconda3\\envs\\dataprog\\lib\\site-packages\\cssutils\\util.py'>

In [9]:
#03-1993/ws-716240

In [10]:
def get_table_header(tb_head):
    head = []
    for cell in tb_head.find_all("th"):
        head.append(cell.text)
    return head

In [11]:
def get_df_from_table(table_header, table_body,span_dict):
    df_data = []
    for row in table_body:
        cell_data = []
        for cell in row.find_all("td"):
            spn = cell.find_all("span")
            if len(spn) > 0:
                text = ''
                for sp in spn:
                    span_class = sp.get('class')[0]
                    text = text + span_dict[span_class]
                cell_data.append(text)
            else:
                cell_data.append(cell.text)
        df_data.append(cell_data)
    df = pd.DataFrame(data= df_data, columns= table_header)
    return df

In [12]:
def clean_df(table_df):
    df = table_df.copy()
    df['RA'] = df['RA'].apply(lambda x: 1 if x=='o' else 0)
    df['SN'] = df['SN'].apply(lambda x: 1 if x=='o' else 0)
    df['TS'] = df['TS'].apply(lambda x: 1 if x=='o' else 0)
    df['FG'] = df['FG'].apply(lambda x: 1 if x=='o' else 0)
    tab_header = df.columns
    tab_data = df.values
    df_data = []
    for row_data in tab_data:
        row = []
        for val in row_data:
            val = str(val)
            if val == '-' or len(val) == 0:
                row.append(np.nan)
            elif val.isdigit():
                row.append(np.int(val))
            else:
                row.append(np.float(val))
        df_data.append(row)
    return pd.DataFrame(data= df_data, columns=tab_header)

In [13]:
def clean_missing_data(df, cols, split_size):
    data_arr = np.array_split(df, split_size)
    dfs = []
    for data in data_arr:
        tdf = pd.DataFrame(data= data, columns= cols)
        mean = tdf.mean()
        tdf.fillna(mean, inplace=True)
        dfs.append(tdf)
    return pd.concat(dfs)

In [14]:
def get_daily_weather_data(city_id, city_name, month, year):
    req = requests.get(URL.format(month, year, city_id))
    soup = BeautifulSoup(req.text, 'html.parser')
    table = soup.findAll('table', {'class': 'medias mensuales numspan'})[0]
    tb = table.find_all("tr")
    tb_len= len(tb)
    tb_head = tb[0]
    tb_body = tb[1:tb_len-2]
    span_dict = get_span_dict(soup)
    table_header = get_table_header(tb_head)
    table_df = get_df_from_table(table_header, tb_body, span_dict)
    df_clean = clean_df(table_df)
    df = clean_missing_data(df_clean, table_header, 7)
    df['Month'] = month
    df['Year'] = year
    df['City'] = city_name
    return df

In [16]:
df = get_daily_weather_data('716240','Toronto','02','2001')

In [17]:
df.head()

,Day,T,TM,Tm,SLP,H,PP,VV,V,VM,VG,RA,SN,TS,FG,Month,Year,City
0,1,0.2,3.5,-2.8,1013.6,73,0.76,19.6,15.7,20.6,59.4,1,1,0,0,02,2001,Toronto
1,2,-2.2,2.5,-7.9,1014.2,71,1.52,16.7,22.6,48.2,59.4,0,1,0,0,02,2001,Toronto
2,3,-9.3,0.2,-13.9,1023.1,64,0.76,23.8,16.5,38.9,59.4,0,1,0,0,02,2001,Toronto
3,4,-2.8,0.2,-6.4,1021.7,69,0.25,17.1,9.8,14.8,59.4,0,1,0,0,02,2001,Toronto
4,5,1.4,2.0,-4.2,1014.3,89,2.54,8.4,9.3,16.5,40.8,1,1,0,1,02,2001,Toronto


In [24]:
def get_weather_data():
    dfs = []
    for city_name, city_id in cities.items():
        for year in range(2009, 2020):
            year = str(year)
            for month in range(1, 13):
                if month < 10:
                    month = '0'+str(month)
                else:
                    month = str(month)
                df = get_daily_weather_data(city_id, city_name, month, year)
                dfs.append(df)
    return pd.concat(dfs)

In [25]:
start_time = time.time()
tdf = get_weather_data()
end_time = time.time()

In [26]:
tdf

,Day,T,TM,Tm,SLP,H,PP,VV,V,VM,VG,RA,SN,TS,FG,Month,Year,City
0,1,-10.9,-5.8,-15.6,1026.5,65.0,0.00,24.0,12.4,40.7,57.6,0,0,0,0,01,2009,Toronto
1,2,-2.1,0.3,-5.4,1008.6,63.0,0.00,22.4,17.8,35.2,40.7,0,0,0,0,01,2009,Toronto
2,3,-4.3,-2.0,-7.0,1017.4,65.0,0.51,24.0,22.8,35.2,48.2,0,0,0,0,01,2009,Toronto
3,4,-7.3,-1.9,-12.2,1022.1,63.0,0.00,22.9,11.3,18.3,35.2,1,0,0,0,01,2009,Toronto
4,5,-2.7,1.0,-7.0,1016.7,65.0,0.00,20.6,20.2,35.2,44.3,1,0,0,0,01,2009,Toronto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,27,-6.2,0.0,-10.2,1025.4,81.0,0.00,22.0,14.3,24.1,29.4,0,1,0,0,12,2019,Halifax
27,28,1.9,4.0,0.0,1010.0,92.0,7.62,18.7,24.3,33.5,44.6,1,1,0,0,12,2019,Halifax
28,29,-0.9,1.0,-3.2,1014.5,68.0,0.00,24.1,23.2,35.2,44.6,0,1,0,0,12,2019,Halifax
29,30,-3.7,-1.0,-6.0,1019.9,78.0,0.00,24.1,20.4,33.5,48.2,0,1,0,0,12,2019,Halifax


In [28]:
tdf.columns

Index(['Day', 'T', 'TM', 'Tm', 'SLP', 'H', 'PP', 'VV', 'V', 'VM', 'VG', 'RA',
       'SN', 'TS', 'FG', 'Month', 'Year', 'City'],
      dtype='object')

In [29]:
col_ordered = ['Day', 'Month', 'Year', 'City', 'T', 'TM', 'Tm', 'SLP', 'H', 'PP', 'VV', 'V', 'VM', 'VG', 'RA',
       'SN', 'TS', 'FG']

In [32]:
ff = tdf[col_ordered]

In [27]:
end_time - start_time 

1839.8811826705933

In [33]:
ff.to_csv('new_final.csv', index=False)